In [46]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import regex as re
import scipy
import seaborn as sns
import matplotlib.pyplot as plt


In [47]:
def get_atsphere_taxonomy(taxonomy_df):
    #Parse the taxonomy table and separate into sphingo and non-sphingo
    #Return dataframe with id and taxonomy
    sphingo_df = taxonomy_df[taxonomy_df.loc[:, 1] == 'Sphingomonadaceae']
    nonsphingo_df = taxonomy_df[taxonomy_df.loc[:, 1] != 'Sphingomonadaceae']

    return sphingo_df, non_sphingo_df

def parse_media_preference():
    #parse the xml table with the media experiment for each strain
    #return dataframe with strain id and media preference
    return

def get_atsphere_media_preference():
    #merge the two dataframes and return the merged dataframe   
    return

In [48]:
def get_at_name(genomes_path, taxonomy_df):
    leaf_genomes = pd.DataFrame(index=taxonomy_df.index, columns=['at_name'])
    for genome in taxonomy_df.index:
#        print (genomes_path.loc[genome]['path'])
        fasta = open(genomes_path.loc[genome]['path']).read()
        header = fasta.split('\n')[0]
        if 'Leaf' in header:
            leaf_name = 'Leaf{}'.format(header.split('Leaf')[1].split(' ')[0])
            if "_" in leaf_name:
                leaf_name = leaf_name.split('_')[0]
            leaf_genomes.loc[genome]['at_name'] = leaf_name

    return leaf_genomes.dropna(how='any')

#get_at_name(taxonomy_df)
genomes_path =  pd.read_csv('/DATA_RAID2/vtracann/shared/db/isolates/genomes_representatives_batchfile.txt', sep='\t', header=None, names=['path', 'genome'])
genomes_path.set_index('genome', inplace=True)
taxonomy_df = pd.read_csv('/DATA_RAID2/vtracann/shared/db/isolates/family_gtdbtk_annot.tsv', sep='\t', index_col=0, header=None)

leaf_genomes = get_at_name(genomes_path, taxonomy_df)



In [75]:

media_preference_file = pd.read_csv('/DATA_RAID2/vtracann/shared/db/isolates/medium/CSourceScreen_Jul2022.csv', sep='\t', index_col=0)

print (media_preference_file.sum(axis=1))


#drop columns with NaNs.
media_preference_file = media_preference_file.dropna(how='any',axis=1)

#sphingo_df, nonsphingo_df = get_atsphere_taxonomy(taxonomy_df)

#print (sphingo_df.index)
print (media_preference_file.index)

#find out why they are not here => wrong field in the metadata file in ncbi
print ([x for x in media_preference_file.index if x not in leaf_genomes.values])


Leaf1      18.0
Leaf2       8.0
Leaf3       9.0
Leaf4      19.0
Leaf5      15.0
           ... 
Leaf159    18.0
Leaf88     11.0
Leaf89     11.0
Leaf106    14.0
Leaf108    11.0
Length: 224, dtype: float64
Index(['Leaf1', 'Leaf2', 'Leaf3', 'Leaf4', 'Leaf5', 'Leaf7', 'Leaf8', 'Leaf9',
       'Leaf10', 'Leaf11',
       ...
       'Leaf466', 'Leaf469', 'Leaf129', 'Leaf130', 'Leaf145', 'Leaf159',
       'Leaf88', 'Leaf89', 'Leaf106', 'Leaf108'],
      dtype='object', length=224)
['Leaf4', 'Leaf5', 'Leaf9', 'Leaf21', 'Leaf29', 'Leaf32', 'Leaf37', 'Leaf59', 'Leaf68', 'Leaf73', 'Leaf98', 'Leaf118', 'Leaf132', 'Leaf167', 'Leaf170', 'Leaf171', 'Leaf185', 'Leaf205', 'Leaf248', 'Leaf250', 'Leaf294', 'Leaf313', 'Leaf335', 'Leaf350', 'Leaf374', 'Leaf401', 'Leaf407', 'Leaf408']


In [50]:
#rename rows in media preference file to match taxonomy file using leaf_genomes to match
leaf_genome_values = leaf_genomes.values.flatten()
media_index = media_preference_file.index
new_index = []

for i in media_index:
    if i in leaf_genomes.values.flatten():
        i_match = np.where(leaf_genomes.values.flatten() == i)[0][0]
        new_index.append(leaf_genomes.index[i_match])
    else:
        new_index.append(i)

media_preference_file.index = new_index

t_media = pd.concat([media_preference_file, taxonomy_df], axis=1).dropna(how='any')

print (t_media)

#remember that there are l-sphere missing for metdata reasons

               h2o  malt  his__L  galur  trp__L  tyr__L  glcn  inost  arab__L   
GCF_001421315  0.0   1.0     1.0    0.0     0.0     0.0   1.0    0.0      0.0  \
GCF_001421325  0.0   0.0     0.0    1.0     0.0     0.0   0.0    0.0      0.0   
GCF_001425965  0.0   1.0     0.0    0.0     0.0     0.0   0.0    0.0      0.0   
GCF_001425985  0.0   0.0     0.0    0.0     0.0     0.0   1.0    1.0      0.0   
GCF_001421165  0.0   1.0     0.0    0.0     0.0     0.0   1.0    1.0      0.0   
...            ...   ...     ...    ...     ...     ...   ...    ...      ...   
GCF_001423615  0.0   1.0     0.0    0.0     0.0     1.0   1.0    0.0      0.0   
GCF_001422795  1.0   0.0     0.0    0.0     0.0     0.0   0.0    0.0      0.0   
GCF_001422215  1.0   0.0     0.0    0.0     0.0     0.0   0.0    0.0      0.0   
GCF_001422895  1.0   0.0     0.0    0.0     0.0     0.0   1.0    0.0      0.0   
GCF_001423265  1.0   0.0     0.0    0.0     0.0     0.0   0.0    0.0      0.0   

               glu__L  ... 

In [95]:
substrate_summary = pd.concat([t_media.iloc[:, :-1].sum(axis=1), t_media.iloc[:, -1]], axis=1)
print (substrate_summary[substrate_summary.loc[:, 1] == 'Weeksellaceae'])
#sns.histplot(substrate_summary, bins=20, hue='1')
#plt.show()

                 0              1
GCF_001424585  0.0  Weeksellaceae
GCF_001421435  0.0  Weeksellaceae
GCF_001424105  0.0  Weeksellaceae
GCF_001424145  0.0  Weeksellaceae
GCF_001425355  0.0  Weeksellaceae


In [97]:
sphingo_list = taxonomy_df.loc[taxonomy_df[1] == 'Sphingomonadaceae'].index.tolist()
nonsphingo_list = taxonomy_df.loc[taxonomy_df[1] != 'Sphingomonadaceae'].index.tolist()

#print (len(sphingo_list))
#print (len(nonsphingo_list))

media_preference_sphingo = t_media[t_media.loc[:, 1]  == 'Sphingomonadaceae']

#print counts of column 1 for each species
#print (t_media.loc[:, 1].value_counts())

abundant_taxa = t_media.loc[:, 1].value_counts().index.tolist()[:9]
#NB: we exclude the last one because weeksellaceae does not grow in the media we use

for taxa in abundant_taxa:
    print (taxa)
    #print (t_media.loc[t_media.loc[:, 1] == taxa])
    #calculate the jaccard index within species and between this species and the rest of the species
    #to do that, we use scipy.signal.correlate for all the columns but the last one (which is the species name) which gives us the jaccard index
    #next, we calculate the mean of the jaccard index within species and between this species and the rest of the species
    #we also calculate the standard deviation of the jaccard index within species and between this species and the rest of the species

    species_media = t_media.loc[t_media.loc[:, 1] == taxa].iloc[:, :-1]
    nonspecies_media = t_media.loc[t_media.loc[:, 1] != taxa].iloc[:, :-1]

    within_species_jaccard_df = pd.DataFrame(index=species_media.index, columns=species_media.index)
    between_species_jaccard_df = pd.DataFrame(index=species_media.index, columns=nonspecies_media.index)

    for i in species_media.index:
        i_tot = species_media.loc[i, :].sum()
        for j in species_media.index:
            if i == j:
                within_species_jaccard_df.loc[i, j] = np.nan
            else:
                j_tot = species_media.loc[j, :].sum()
                within_species_jaccard_df.loc[i, j] = scipy.signal.correlate(species_media.loc[i, :], species_media.loc[j, :], mode='valid')[0]/min(i_tot, j_tot)
        for k in nonspecies_media.index:
            k_tot = nonspecies_media.loc[k, :].sum()
            between_species_jaccard_df.loc[i, k] = scipy.signal.correlate(species_media.loc[i, :], nonspecies_media.loc[k, :], mode='valid')[0]/min(i_tot, k_tot)
        
    #create a histplot where we show both the within species and between species jaccard index distributions

    within_species_jaccard_df = within_species_jaccard_df.astype(float)
    between_species_jaccard_df = between_species_jaccard_df.astype(float)
    
    within_species_jaccard_df = within_species_jaccard_df.stack().reset_index()
    within_species_jaccard_df.columns = ['species1', 'species2', 'jaccard_index']

    between_species_jaccard_df = between_species_jaccard_df.stack().reset_index()
    between_species_jaccard_df.columns = ['species1', 'species2', 'jaccard_index']

    within_species_jaccard_df['type'] = 'within_species'
    between_species_jaccard_df['type'] = 'between_species'
    jaccard_df = pd.concat([within_species_jaccard_df, between_species_jaccard_df])

    fig, ax = plt.subplots(figsize=(10, 10))
    sns.violinplot(data=jaccard_df, x='jaccard_index', y='type', ax=ax, bw_adjust=5)
    ax.set_title(taxa)
    plt.show()
    plt.close()

    print ('mean within species jaccard index: ' + str(within_species_jaccard_df['jaccard_index'].mean()))
    print ('std within species jaccard index: ' + str(within_species_jaccard_df['jaccard_index'].std()))
    print ('mean between species jaccard index: ' + str(between_species_jaccard_df['jaccard_index'].mean()))
    print ('std between species jaccard index: ' + str(between_species_jaccard_df['jaccard_index'].std()))






['Microbacteriaceae', 'Beijerinckiaceae', 'Sphingomonadaceae', 'Rhizobiaceae', 'Burkholderiaceae', 'Pseudomonadaceae', 'Nocardioidaceae', 'Mycobacteriaceae', 'Micrococcaceae']
Microbacteriaceae


/tmp/ipykernel_3145126/738855799.py:38: RuntimeWarning: invalid value encountered in scalar divide
  between_species_jaccard_df.loc[i, k] = scipy.signal.correlate(species_media.loc[i, :], nonspecies_media.loc[k, :], mode='valid')[0]/min(i_tot, k_tot)


Beijerinckiaceae


/tmp/ipykernel_3145126/738855799.py:38: RuntimeWarning: invalid value encountered in scalar divide
  between_species_jaccard_df.loc[i, k] = scipy.signal.correlate(species_media.loc[i, :], nonspecies_media.loc[k, :], mode='valid')[0]/min(i_tot, k_tot)


Sphingomonadaceae


...

/tmp/ipykernel_3145126/738855799.py:38: RuntimeWarning: invalid value encountered in scalar divide
  between_species_jaccard_df.loc[i, k] = scipy.signal.correlate(species_media.loc[i, :], nonspecies_media.loc[k, :], mode='valid')[0]/min(i_tot, k_tot)


Mycobacteriaceae


/tmp/ipykernel_3145126/738855799.py:38: RuntimeWarning: invalid value encountered in scalar divide
  between_species_jaccard_df.loc[i, k] = scipy.signal.correlate(species_media.loc[i, :], nonspecies_media.loc[k, :], mode='valid')[0]/min(i_tot, k_tot)


Micrococcaceae


/tmp/ipykernel_3145126/738855799.py:38: RuntimeWarning: invalid value encountered in scalar divide
  between_species_jaccard_df.loc[i, k] = scipy.signal.correlate(species_media.loc[i, :], nonspecies_media.loc[k, :], mode='valid')[0]/min(i_tot, k_tot)
